In [2]:
import graphlab as gl

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.3.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [3]:
import numpy as np
import scipy as sp

# 1.Reading Data

In [4]:
features = gl.SFrame.read_csv("features.csv")

[INFO] GraphLab Create v1.8.3 started. Logging: /tmp/graphlab_server_1458801396.log


Finished parsing file /Users/sergey/Documents/Coursera/HSE/kaggle/features.csv

Parsing completed. Parsed 97230 lines in 5.56761 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,float,float,float,str,float,float,float,int,int,int,int,float,float,float,float,int,int,int,int,float,int,int,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [5]:
features_check = gl.SFrame.read_csv("features_test.csv")

Read 100 lines. Lines per second: 161.843

Finished parsing file /Users/sergey/Documents/Coursera/HSE/kaggle/features_test.csv

Parsing completed. Parsed 100 lines in 0.619302 secs.

Finished parsing file /Users/sergey/Documents/Coursera/HSE/kaggle/features_test.csv

Parsing completed. Parsed 17177 lines in 0.730097 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,float,float,float,str,float,float,float,int,int,int,int,float,float,float,float,int,int,int,int,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [6]:
#features_check.print_rows(num_rows=3, num_columns=109)

# 2. Removing irrelevant columns

In [7]:
name_vector = features_check.column_names()[2:]

# 3. Reshaping data

# 3. Splitting train and test

In [8]:
features_train, features_test = features.random_split(.9, seed=5)

In [9]:
y_train = features_train['radiant_win']
y_test = features_test['radiant_win']

In [10]:
X_test = features_test[name_vector]
X_train = features_train[name_vector]

In [11]:
#X_test.print_rows(num_rows=3, num_columns=109)

# 4. Analysing and filling NAs

In [12]:
na_names = list()

In [13]:
for f in name_vector:
    N = sum(X_train[f] == None)
    if N > 0:
        na_names.extend([f])
        print f," contains NA:", sum(X_train[f] == None)

first_blood_time  contains NA: 17589
first_blood_team  contains NA: 17589
first_blood_player1  contains NA: 17589
radiant_bottle_time  contains NA: 14141
radiant_courier_time  contains NA: 618
radiant_flying_courier_time  contains NA: 24696
radiant_first_ward_time  contains NA: 1625
dire_bottle_time  contains NA: 14502
dire_courier_time  contains NA: 610
dire_flying_courier_time  contains NA: 23425
dire_first_ward_time  contains NA: 1634


In [14]:
for f in na_names:
    print f
    X_train = X_train.fillna(f, 0)
    X_test = X_test.fillna(f, 0)

first_blood_time
first_blood_team
first_blood_player1
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


In [15]:
for f in name_vector:
    N = sum(X_test[f] == None)
    if N > 0:
        print f," contains NA:", sum(X_test[f] == None)

# 5. Logistics regression

In [16]:
X_test['target'] = y_test
X_train['target'] = y_train

In [17]:
model1 = gl.logistic_classifier.create(X_train, target='target', l2_penalty = 0.0001)

Logistic regression:

--------------------------------------------------------

Number of examples          : 82895

Number of classes           : 2

Number of feature columns   : 101

...

| 3         | 4        | 22.323549    | 0.655818          | 0.657808            |

| 4         | 5        | 29.236253    | 0.655818          | 0.657808            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



In [18]:
y_pred1 = model1.predict(X_test)

In [19]:
gl.evaluation.roc_curve(y_test, y_pred1)
gl.evaluation.accuracy(y_test, y_pred1)

0.6531384175015335

# 6. Gradient boosting

In [20]:
model2 = gl.boosted_trees_classifier.create(X_train, target='target', max_iterations=50, max_depth = 5, random_seed=1)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 83231

Number of classes           : 2

Number of feature columns   : 101

...

| 48        | 48.598319    | 0.701073          | 0.642874            |

| 49        | 49.517203    | 0.702214          | 0.643348            |

| 50        | 50.347256    | 0.703392          | 0.64074             |

+-----------+--------------+-------------------+---------------------+

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



In [21]:
y_pred2 = model2.predict(X_test)
gl.evaluation.roc_curve(y_test, y_pred2)
gl.evaluation.accuracy(y_test, y_pred2)

0.6454712737681456

# 7. Random forests

In [22]:
model3 = gl.random_forest_classifier.create(X_train, target='target', num_trees = 10)

Random forest classifier:

--------------------------------------------------------

Number of examples          : 83112

Number of classes           : 2

Number of feature columns   : 101

...

| 8         | 8.519604     | 0.633134          | 0.619926            |

| 9         | 9.372556     | 0.633892          | 0.618312            |

| 10        | 10.199607    | 0.633711          | 0.621541            |

+-----------+--------------+-------------------+---------------------+

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



In [23]:
y_pred3 = model3.predict(X_test)
gl.evaluation.accuracy(y_test, y_pred3)

0.6173584134123901

In [24]:
gl.evaluation.accuracy(y_pred2, y_pred1)

0.8541198118994071

# 8.Combine classifiers

In [40]:
z = gl.SFrame()
t = gl.SFrame()

In [41]:
z['target'] = y_train

In [42]:
z['feat1'] = model1.predict(X_train)
z['feat2'] = model2.predict(X_train)

In [43]:
model_f = gl.logistic_classifier.create(z, target='target')

Logistic regression:

--------------------------------------------------------

Number of examples          : 82979

Number of classes           : 2

Number of feature columns   : 2

...

| 2         | 3        | 0.481533     | 0.700370          | 0.700380            |

| 3         | 4        | 0.697483     | 0.700370          | 0.700380            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



In [44]:
t['feat1'] = model1.predict(X_test)
t['feat2'] = model2.predict(X_test)

In [45]:
t['pred'] = model_f.predict(t)

In [46]:
gl.evaluation.accuracy(y_test, t['pred'])

0.6454712737681456